In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import glob
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern Roman'], 'size':14})
rc('text', usetex=True)

In [8]:
kaluz = pd.read_csv('Kaluzny2004rrl.txt', delim_whitespace=True)
for thing in glob.glob('Omega_Cen_phot/*.txt'):
    starid = int(thing.split('/')[1].split('.')[0])
    jhk = np.loadtxt(thing)
    b = jhk[jhk[:,3] == 3]
    v = jhk[jhk[:,3] == 2]
    j = jhk[jhk[:,3] == -5]
    h = jhk[jhk[:,3] == -1]
    k = jhk[jhk[:,3] == -2]
    per = kaluz.per[kaluz.id == starid].values[0]
    all_out = []
    for index,i in {'3':b,'5':v,'11':j,'13':h,'15':k}.iteritems():
        if i.shape[0] > 0:
            g_out = np.zeros((len(i),25))
            g_out[:,0] = i[:,0]
            g_out[:,2::2] += 9.9999
            g_out[:,1::2] += 99.999
            g_out[:,int(index)] = i[:,1]
            g_out[:,int(index)+1] = i[:,2]
            all_out.append(g_out)
    output = np.vstack(all_out)
    if os.path.exists('irac/{}.gloess_in'.format(starid)):
        irac = np.loadtxt('irac/{}.gloess_in'.format(starid), skiprows=4)
        output = np.vstack([output, irac])
    f = open('{}_multi.gloess_in'.format(starid), 'w')
    f.write('{}\n'.format(starid))
    f.write('{}\n'.format(per))
    f.write('{}\n'.format(len(output)))
    f.write('0.10 0.10 0.10 0.10 0.20 0.20 0.20 0.20 0.20 0.20 0.20 0.20\n')
    for line in output:
        f.write(' '.join(map(str, line)) + '\n')
    f.close()

In [22]:
irac = np.loadtxt('3.gloess_in', skiprows=4)
three = irac[irac[:,17] != 99.999]
three[:,17] = 99.999
three[:,18] = 9.9999
j_out = three.copy()
j_out[:,0] = j[:,0]
j_out[:,11] = j[:,1]
j_out[:,12] = j[:,2]
h_out = three.copy()
h_out[:,0] = h[:,0]
h_out[:,13] = h[:,1]
h_out[:,14] = h[:,2]
k_out = three.copy()
k_out[:,0] = k[:,0]
k_out[:,15] = k[:,1]
k_out[:,16] = k[:,2]
out = np.vstack([irac, j_out, h_out, k_out])

In [25]:
f = open('3.gloess_in')
head = list(f)[:4]
f.close()
head
f = open('multiband_test.gloess_in', 'w')
f.write(''.join(head))
for i in out:
    f.write(' '.join(map(str, i)) + '\n')
f.close()